# Familial DistilBERT Model Using Single Batch 1 + Batch 2

In [5]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from ktrain import text
import random
import warnings
from sklearn.utils import shuffle

# Set random seed
random.seed(18)
seed = 18

# Ignore warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_colwidth', None)

## 1. Loading the data and quick exploratory data analysis

In [ ]:
# import os

# # Define the folder path and themes
# folder_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method'
# themes = [
#     'Aspirational', 'Attainment', 'Community Consciousness', 'Familial', 'Filial Piety', 
#     'First Gen', 'Navigational', 'Perseverance', 'Resistance', 'Social', 'Spiritual'
# ]

# # Initialize an empty dictionary to store DataFrames
# batch_1_theme_dataframes = {}
# # Loop through each theme and load its corresponding file
# for theme in themes:
#     # Construct the filename without modifying the theme name
#     file_name = f"{theme}_sentence_level_batch_1_jaccard.csv"
#     file_path = os.path.join(folder_path, file_name)
    
#     # Check if the file exists before attempting to load
#     if os.path.exists(file_path):
#         batch_1_theme_dataframes[theme] = pd.read_csv(file_path)
#         print(f"Loaded {file_name}")
#     else:
#         print(f"File not found for theme: {theme}")

# # Define the folder path and themes
# folder_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method/batch_2'
# themes = [
#     'Aspirational', 'Attainment', 'Community Consciouss', 'Familial', 'Filial Piety', 
#     'First Generation', 'Navigational', 'Perseverance', 'Resistance', 'Social', 'Spiritual'
# ]

# # Initialize an empty dictionary to store DataFrames
# batch_2_theme_dataframes = {}
# # Loop through each theme and load its corresponding file
# for theme in themes:
#     # Construct the filename without modifying the theme name
#     file_name = f"{theme}_sentence_level_batch_2_jaccard.csv"
#     file_path = os.path.join(folder_path, file_name)
    
#     # Check if the file exists before attempting to load
#     if os.path.exists(file_path):
#         batch_2_theme_dataframes[theme] = pd.read_csv(file_path)
#         print(f"Loaded {file_name}")
#     else:
#         print(f"File not found for theme: {theme}")

In [29]:

merged_familial_df = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/corrections/Familial/Familial_Merged_evaluated.csv")



In [30]:

# # Train-test split
training_df, test_df = train_test_split(merged_familial_df, test_size=0.1, random_state=18, stratify=merged_familial_df['label'])

In [31]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (1899, 3) 
Test dataset shape: (211, 3)
Positive labels present in the dataset : 391  out of 1899 or 20.5897840968931%
Positive labels present in the test dataset : 43  out of 211 or 20.379146919431278%


## 2. Experimental Design

In [26]:
# MAXLEN = 256

# X_train = training_df['sentence']
# y_train = training_df['label']
# X_test = validate_df['sentence']
# y_test = validate_df['label']
# # Split the data
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 18, stratify=y)
# # X_test.shape

# model_name = 'bert-base-uncased'

# distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
# training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
# validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
# distillbert_base_model = distillbert_transformer.get_classifier()

preprocessing train...
language: en
train sequence lengths:
	mean : 22
	95percentile : 41
	99percentile : 60


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 22
	95percentile : 39
	99percentile : 58


In [32]:
MAXLEN = 150

X = training_df['sentence']
y = training_df['label']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 18, stratify=y)


model_name = 'bert-base-uncased'

distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 42
	99percentile : 58


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 43
	99percentile : 66


In [33]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights

class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = dict(zip(classes, class_weights))

[0.62964191 2.42838875]


In [ ]:
# Build BERT model
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)
distillbert_learner.set_weight_decay(0.100)
distillbert_learner.autofit(0.000020, epochs=12, early_stopping=4, class_weight=class_weights)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/12
285/285 [==============================] - 179s 567ms/step - loss: 0.4105 - accuracy: 0.7273 - val_loss: 0.1361 - val_accuracy: 0.9684
Epoch 2/12
 60/285 [=====>........................] - ETA: 2:47 - loss: 0.0895 - accuracy: 0.9722

In [71]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

8/8 [==============================] - 13s 746ms/step
              precision    recall  f1-score   support

           0       0.99      0.86      0.92       196
           1       0.53      0.97      0.68        32

    accuracy                           0.87       228
   macro avg       0.76      0.91      0.80       228
weighted avg       0.93      0.87      0.89       228



array([[168,  28],
       [  1,  31]])

In [72]:
distillbert_learner.model.summary()

Model: "tf_bert_for_sequence_classification_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_303 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [74]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [75]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [76]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [77]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 197, False Positive: 21, False Negative: 2, True Positive: 33


In [78]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      0.90      0.94       218
         1.0       0.61      0.94      0.74        35

    accuracy                           0.91       253
   macro avg       0.80      0.92      0.84       253
weighted avg       0.94      0.91      0.92       253
 



In [58]:
test_df_copy = test_df.copy()
test_df_copy["predicted_label"] = y_pred_distillbert
test_df_copy

,sentence,label,phrase,predicted_label
0,every time i have to spend more effort and time to understand the physics concept and finish my homework.,0,"['I am an International student from China, and my parents send me here because they want me to have better education resources.']",0
1,there is a big need for people of color in the medical field and i want to start to make a difference.,0,"[""I am also here to advance my education and make my family proud. I'll be the first in my family to graduate from college and to attend college in general but I want to make them proud.""]",0
2,i enjoy the process of breaking down problems to a point that they can be solved.,0,['I decided to follow my parents lead.'],0
3,"furthermore, i am here learning physics because it is not only a requirement for my major, but because i am excited to learn the subject itself and how it applies to our everyday lives.",0,['I am here because I want to honor the sacrifices that both my parents had to take in order for me to get an education here in the United States.'],0
4,this class was a requirement in my major which is why i am here taking it.,0,['I wouldnt be here without my parents who sacrificed a lot for me to be here. I am part of the first generation in my family to go to college and hopefully get a degree and I am trying to make my parents proud but also learn things for myself and get a career that will give me financial stability and also a career where I will help tons of people.'],0
...,...,...,...,...
250,"when i graduated middle school in china, my parents decided send me to the united state to finish my high school and university study.",1,"['When I graduated middle school in China, my parents decided send me to The United State to finish My high school and university study.']",1
251,my aunt is someone i look up to since she is a radiologist at the stanford medical center in palo alto.,0,['My aunt is someone I look up to since she is a radiologist at the Stanford Medical Center in Palo Alto. I want to be able to follow in her steps and to make my family and her proud. Shes someone who highly supports me getting a college education and she is one of the reasons I continue to push myself even when things get difficult.'],0
252,i am here because i care about my community and to help empower those around me.,0,"[""become a role model for my younger cousins back where I'm from.""]",0
253,"in addition to this, i do hope to someday make a difference in the world, and strongly believe that i can do so if i pursue a stem related career.",0,"['I come from an Indian immigrant family, and all of my family members have worked hard to earn their places in the world. Most of my family members are successful, and a majority of them are working in STEM related careers. Due to this, there has always been an expectation for me to go into a STEM related field.']",0


In [59]:
test_df_copy.to_excel("Familial_error_analysis_2.xlsx")

In [24]:
# distillbert_predictor.save('../../model/first_generation_distilbert_base_uncased_model_10102020') # 256 MB

In [21]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.9306334080717489


In [22]:
from sklearn.metrics import auc, precision_recall_curve

# Replace these with your actual data
# distillbert_test_label: Ground truth labels (list or numpy array)
# y_pred_distillbert: Predicted probabilities (list or numpy array)

# Calculate precision-recall curve
precision, recall, _ = precision_recall_curve(distillbert_test_label, y_pred_distillbert)

# Calculate PR AUC
pr_auc_score = auc(recall, precision)

# Print the result
print("AUC PR score for DistilBERT model: ", pr_auc_score)

AUC PR score for DistilBERT model:  0.7918205047976637
